# Dataset Processing for Visualisation

Custom Python scripts to process and convert dataset from CSV format to JSON for better integrate with D3.

In [10]:
import pandas as pd

# Load the CSV file into a DataFrame
dataset = pd.read_csv('/Users/nick/Documents/GitHub/COS30045-Assignment3/data/visualisation2/Edited_Dataset.csv')

# Display the first few rows of the DataFrame
dataset.head()

,country,cause,year,total
0,Argentina,Accidental falls,1982,10.4
1,Argentina,Accidental poisoning,1982,1.2
2,Argentina,Accidents,1982,48.9
3,Argentina,Acute myocardial infarction,1982,115.9
4,Argentina,Assault,1982,4.8


In [24]:
dataset = '/Users/nick/Documents/GitHub/COS30045-Assignment3/data/visualisation2/Edited_Dataset.csv'

output = '/Users/nick/Documents/GitHub/COS30045-Assignment3/data/visualisation2/Output.json'

In [32]:
import pandas as pd
import json

# Load the CSV data
csv_file = dataset  # Update the path to your file
df = pd.read_csv(csv_file)

# Function to replace unicode single quotation marks with regular apostrophes
def clean_cause_name(cause_name):
    return cause_name.replace("\u2019", "'")

# Define the full hierarchy of causes based on the structure you provided
full_structure = {
    "Certain infectious and parasitic diseases": {
        "Tuberculosis": [],
        "HIV-AIDS": []
    },
    "Neoplasms": {
        "Malignant neoplasms": [
            "Malignant neoplasms of trachea, bronchus, lung",
            "Malignant neoplasms of colon, rectum and anus",
            "Malignant neoplasms of stomach",
            "Malignant neoplasms of pancreas",
            "Malignant neoplasms of liver",
            clean_cause_name("Hodgkin’s disease"),
            "Leukemia",
            "Malignant neoplasms of bladder",
            "Malignant melanoma of skin"
        ]
    },
    "Diseases of the blood and blood-forming organs": {},
    "Endocrine, nutritional and metabolic diseases": {
        "Diabetes mellitus": []
    },
    "Mental and behavioural disorders": {
        "Dementia": []
    },
    "Diseases of the nervous system": {
        clean_cause_name("Parkinson’s disease"): [],
        clean_cause_name("Alzheimer’s disease"): []
    },
    "Diseases of the respiratory system": {
        "Influenza": [],
        "Pneumonia": [],
        "Chronic obstructive pulmonary diseases": [],
        "Asthma": []
    },
    "Diseases of the circulatory system": {
        "Ischaemic heart diseases": [
            "Acute myocardial infarction"
        ],
        "Cerebrovascular diseases": []
    },
    "Diseases of the digestive system": {
        "Peptic ulcer": [],
        "Chronic liver diseases and cirrhosis": []
    },
    "Diseases of the skin and subcutaneous tissue": {},
    "Diseases of the musculoskeletal system and connective tissue": {},
    "Diseases of the genitourinary system": {},
    "Certain conditions originating in the perinatal period": {},
    "Congenital malformations, deformations and chromosomal abnormalities": {},
    "Symptoms, signs, ill-defined causes": {},
    "External causes of mortality": {
        "Accidents": [
            "Transport Accidents",
            "Accidental falls",
            "Accidental poisoning"
        ],
        "Intentional self-harm": [],
        "Assault": []
    },
    "Codes for special purposes: COVID-19": {}
}

# Initialize an empty dictionary for the JSON output
output_json = {}

# Iterate over the data, grouping by country and year
for country, country_group in df.groupby('country'):
    output_json[country] = {"years": []}
    
    for year, year_group in country_group.groupby('year'):
        # Exclude rows where cause == 'Total'
        filtered_year_group = year_group[year_group['cause'] != 'Total']
        
        # Extract the total deaths for the year where cause == 'Total'
        total_deaths_group = year_group[year_group['cause'] == 'Total']
        if not total_deaths_group.empty:
            total_deaths = total_deaths_group['total'].values[0]
        else:
            total_deaths = filtered_year_group['total'].sum()  # Use the sum if "Total" isn't explicitly present
        
        # Create the year structure without including 'Total' in the causes
        year_data = {
            "year": year,
            "total": total_deaths,  # Total deaths for the year
            "causes": []
        }
        
        # Iterate through the full hierarchy of causes
        for major_cause, sub_causes in full_structure.items():
            cause_data = {
                "cause": major_cause,
                "total": 0  # Default total if not present
            }
            
            # Check if the major cause exists in the current group
            major_cause_group = filtered_year_group[filtered_year_group['cause'] == major_cause]
            if not major_cause_group.empty:
                cause_data["total"] = major_cause_group['total'].values[0]
            
            # Process subCauses only if they exist
            if sub_causes:
                cause_data["subCauses"] = []
                
                sub_cause_sum = 0  # To track the sum of subCauses

                for sub_cause, sub_sub_causes in sub_causes.items():
                    sub_cause_data = {
                        "cause": sub_cause,
                        "total": 0  # Default total if not present
                    }
                    
                    # Check if the sub-cause exists in the current group
                    sub_cause_group = filtered_year_group[filtered_year_group['cause'] == sub_cause]
                    if not sub_cause_group.empty:
                        sub_cause_total = sub_cause_group['total'].values[0]
                        sub_cause_data["total"] = sub_cause_total
                        sub_cause_sum += sub_cause_total
                    
                    # Process subSubCauses if they exist
                    if sub_sub_causes:
                        sub_cause_data["subSubCauses"] = []
                        sub_sub_cause_sum = 0  # To track the sum of subSubCauses
                        
                        for sub_sub_cause in sub_sub_causes:
                            sub_sub_cause_data = {
                                "cause": sub_sub_cause,
                                "total": 0  # Default total if not present
                            }
                            
                            # Check if the sub-sub-cause exists in the current group
                            sub_sub_cause_group = filtered_year_group[filtered_year_group['cause'] == sub_sub_cause]
                            if not sub_sub_cause_group.empty:
                                sub_sub_cause_total = sub_sub_cause_group['total'].values[0]
                                sub_sub_cause_data["total"] = sub_sub_cause_total
                                sub_sub_cause_sum += sub_sub_cause_total
                            
                            sub_cause_data["subSubCauses"].append(sub_sub_cause_data)
                        
                        # If there's a difference between the subCause total and the sum of subSubCauses,
                        # create a subCause-unspecified entry.
                        if sub_sub_cause_sum < sub_cause_data["total"]:
                            unspecified_sub_sub_total = round(sub_cause_data["total"] - sub_sub_cause_sum, 1)
                            sub_cause_data["subSubCauses"].append({
                                "cause": f"{sub_cause}-Unspecified",
                                "total": unspecified_sub_sub_total
                            })
                    
                    cause_data["subCauses"].append(sub_cause_data)
                
                # If there's a difference between the major cause total and the sum of subCauses,
                # create a cause-unspecified entry.
                if sub_cause_sum < cause_data["total"]:
                    unspecified_total = round(cause_data["total"] - sub_cause_sum, 1)
                    cause_data["subCauses"].append({
                        "cause": f"{major_cause}-Unspecified",
                        "total": unspecified_total
                    })
            
            year_data["causes"].append(cause_data)
        
        output_json[country]["years"].append(year_data)

# Output to JSON
output_file = output  # Update the path to your output file
with open(output_file, 'w') as f:
    json.dump(output_json, f, indent=4)

print(f"JSON data successfully written to {output_file}")

JSON data successfully written to /Users/nick/Documents/GitHub/COS30045-Assignment3/data/visualisation2/Output.json


In [36]:
import pandas as pd
import json

# Load the CSV data
csv_file = dataset  # Update the path to your file
df_csv = pd.read_csv(csv_file)

# Load the JSON data
json_file = output  # Update the path to your file
with open(json_file, 'r', encoding='utf-8') as f:
    json_data = json.load(f)

# Function to clean text by replacing \u2019 with a regular apostrophe (if needed)
def clean_text(text):
    return text.replace("\u2019", "'")

# Iterate through the JSON data and compare it to the CSV
errors = []

for country, country_data in json_data.items():
    for year_data in country_data['years']:
        year = year_data['year']
        for cause_data in year_data['causes']:
            cause = clean_text(cause_data['cause'])  # Clean cause names if necessary
            total = cause_data['total']

            # Skip Unspecified causes
            if '-unspecified' in cause.lower():
                continue

            # Special check for COVID-19 cause: Only compare for 2019 onwards
            if cause == "Codes for special purposes: COVID-19" and year < 2020:
                # Skip comparisons for COVID-19 in years before 2019
                continue

            # Check if the same country, year, cause, and total exist in the CSV
            matching_row = df_csv[
                (df_csv['country'] == country) &
                (df_csv['year'] == year) &
                (df_csv['cause'] == cause)
            ]

            # If the JSON total is 0 and no matching row in CSV, it's acceptable
            if total == 0 and matching_row.empty:
                continue

            if matching_row.empty:
                errors.append(f"JSON value not found in CSV - Country: {country}, Year: {year}, Cause: {cause}, Total (JSON): {total}")
            else:
                # Compare totals
                csv_total = matching_row['total'].values[0]  # Get the first matching total from CSV
                if csv_total != total:
                    errors.append(f"Mismatch - Country: {country}, Year: {year}, Cause: {cause}, Total (JSON): {total}, Total (CSV): {csv_total}")

# Print the results
if errors:
    print("Found the following mismatches between JSON and CSV:")
    for error in errors:
        print(error)
else:
    print("All entries match between the JSON and CSV!")

All entries match between the JSON and CSV!
